<a href="https://colab.research.google.com/github/nhwhite212/DealingwithDataSpring2021/blob/colab/G3_Citibike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip3 -q install mysqlclient
!pip3 -q install sql_magic
!pip3 -q install sqlalchemy

In [ ]:
# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'

# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

In [ ]:
# We fetch for now just the time-invariant data
results = requests.get(url_stations).json() 

In [ ]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [ ]:
# We will not be using dataframes for this insertion task. (See the G2 notebook if you want to use Pandas)
# We just put the data in a dataframe to understand what is going on.
import pandas as pd
df = pd.DataFrame(data)
df.head(5)

In [ ]:
# Now, let's connect to our database, where we will store our data.

from sqlalchemy import create_engine
#
# Note, the user can use their own userid and database
# i.e DealingS21GBx   where x is your team number...

conn_string_big = 'mysql://{user}:{password}@{host}/'.format(
    user='DealingS21',
    password='DealingS21!!',
    host='bigdata.stern.nyu.edu',
    port=3306,
    db='DealingS21',
    encoding='utf-8')


engine_bigdata = create_engine(conn_string_big)


In [ ]:
!sudo pip -q install sql_magic

In [ ]:
%reload_ext sql_magic

In [ ]:
%config SQL.conn_name='engine_bigdata'


In [ ]:
from sqlalchemy import create_engine

In [ ]:
%reload_ext sql_magic

In [ ]:
%config SQL.conn_name = 'engine_bigdata'

In [ ]:
%%read_sql
show databases

In [ ]:
# This will be the name of our database.
db_name = 'citibike'

In [ ]:
# Just bookkeeping. Drop the database if it is already there
create_db_query = "DROP DATABASE IF EXISTS {db}".format(db=db_name)
engine_bigdata.execute(create_db_query)

In [ ]:
# Run a query to create a database that will hold the data
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
engine_bigdata.execute(create_db_query)

In [ ]:
# Create the two tables. One for storing the time-invariant station data
# and another table to store the time-varying station status data
table_name = 'Stations'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (station_id int, 
                                name varchar(250), 
                                capacity int,
                                lat float,
                                lon float,
                                region_id int,
                                short_name varchar(250),
                                rental_url varchar(250),
                                eightd_has_key_dispenser bool,
                                PRIMARY KEY(station_id)
                                )'''.format(db=db_name, table=table_name)
engine_bigdata.execute(create_table_query)

In [ ]:
results = requests.get(url_stations).json() 
data = results["data"]["stations"]
data[0]

In [ ]:
# We fetch for now just the time-invariant data
# Notice that we have the INSERT IGNORE so that even when we add the same entry
# again, we do not get an error that the line exists. We do get warnings
# but this is expected

table_name = 'Stations'
query_template = '''INSERT IGNORE INTO {db}.{table}(station_id, 
                                        name,
                                        capacity, 
                                        lat, 
                                        lon,
                                        region_id,
                                        short_name,
                                        rental_url,
                                        eightd_has_key_dispenser) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

for entry in data:
    station_id = int(entry['station_id'])
    name = entry['name']
    capacity = entry['capacity']
    lat = entry['lat']
    lon = entry['lon']
    region_id = entry.get('region_id')
    short_name = entry['short_name']
    rental_url = entry['rental_url']
    eightd_has_key_dispenser = entry['eightd_has_key_dispenser']
                       
    print("Inserting station", station_id, "at", name)
    query_parameters = (station_id, name, capacity, lat, lon,
                        region_id, short_name, rental_url, eightd_has_key_dispenser)
    engine_bigdata.execute(query_template, query_parameters)


In [ ]:
check = pd.read_sql("SELECT * FROM citibike.Stations ", con=engine_bigdata)
# Only print the first 21
check[0:20]

### plot the location of the stations, use the lon and lat and circles of size 10

In [ ]:
%matplotlib inline
check.plot(kind='scatter', x='lon', y='lat', s=10, figsize=(10,10))

In [ ]:
# Create the time-varying table
table_name = 'Status'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (station_id int, 
                                last_reported datetime,
                                num_bikes_available int,
                                num_bikes_disabled int,
                                num_docks_available int,
                                num_docks_disabled int,
                                is_installed bool,
                                is_renting bool,
                                is_returning bool,
                                eightd_has_available_keys bool,
                                PRIMARY KEY(station_id, last_reported)
                                )'''.format(db=db_name, table=table_name)
engine_bigdata.execute(create_table_query)

In [ ]:
results = requests.get(url_status).json() 
data = results["data"]["stations"]
data[0]

In [ ]:
# Now we fetch the data about the time varying elements of the citibike stations
from datetime import datetime

table_name = 'Status'
query_template = '''INSERT IGNORE INTO {db}.{table}(station_id, 
                                            num_bikes_available,
                                            num_bikes_disabled,
                                            num_docks_available,
                                            num_docks_disabled,
                                            is_installed,
                                            is_renting,
                                            is_returning,
                                            last_reported,
                                            eightd_has_available_keys) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

for entry in data:
    #print(entry)
    station_id = int(entry['station_id'])
    num_bikes_available = entry['num_bikes_available']
    num_bikes_disabled = entry['num_bikes_disabled']
    num_docks_available = entry['num_docks_available']
    num_docks_disabled = entry['num_docks_disabled']
    is_installed = entry['is_installed']
    is_renting = entry['is_renting']
    is_returning = entry['is_returning']
    last_reported = datetime.fromtimestamp(entry['last_reported']) 
    eightd_has_available_keys = entry['eightd_has_available_keys']
                       
    #print("Inserting station", station_id)
    query_parameters = (station_id, num_bikes_available, num_bikes_disabled,
                        num_docks_available, num_docks_disabled, is_installed, is_renting,
                        is_returning, last_reported, eightd_has_available_keys)
    engine_bigdata.execute(query_template, query_parameters)



In [ ]:
check = pd.read_sql("SELECT * FROM citibike.Status", con=engine_bigdata)
check


In [ ]:
con.close()